# PDG API tutorial, PyHEP 2024

## Introduction

This is a whirlwind tour of the new PDG Python API!

## Installation

(Optional) First, create a virtual environment and activate it:

```bash
python -m venv ~/pdg.venv
source ~/pdg.venv/bin/activate
```

Now install the package:

```bash
pip install pdg
```

## Preamble

Unless otherwise noted, we assume the following preamble has been run:

In [1]:
import pdg

api = pdg.connect()

## Concepts 

### Editions

There are multiple editions of the PDG database corresponding to the (bi)annual PDG releases. There is also a pdgall.sqlite containing data from all historical editions.

### PDG IDs and nodes 

Here, *PDG IDs* are used to to identify particles, properties of particles, and other quantities of interest. PDG IDs can have parent PDG IDs; for example, the parent PDG ID for a particle's mass will be the PDG ID of the particle itself. Top-level PDG IDs, such as those of particles, generally take the form of a letter followed by three digits. The PDG ID for a particle property takes the form of the parent PDG ID, a period, then some digits. A specific edition can be appended with a slash.

Unlike Monte Carlo IDs, which always identify specific states, a PDG ID can represent multiple states from a multiplet.

#### The `get` function

### Monte Carlo IDs 

Monte Carlo IDs (oftened called "PDG IDs" in the community) are used in MC generators and identify specific states. They are assigned algorithmically, with some historical special cases.

### Basic particle properties

Basic properties, particularly quantum numbers, are directly associated to particles.

### Summary table values 

Measured quantities, such as particle properties, are summarized by the PDG in *summary tables*, which list the PDG's evaluation (e.g. an average or fit) of the quantity based on measurements in the literature. The individual measurements are listed in *listings*. Currently, the API provides access to summary table values. Support for the listings will be added in the future.

A given PDG ID may have multiple summary table values, corresponding, for example, to different measurement techniques.

#### Parent PDG IDs 

A particle property listed in the summary tables will have the particle itself as its parent PDG ID.

#### Sort order

Quantities may have multiple summary table values, in which case the database specifies their ordering.

#### Flags

Additionally, when there are multiple summary table values for a quantity, they may use flags???? Or is this in pdgid/?????

#### "Best" summaries

When multiple values are available, the API can provide the "best" one, i.e. the first-sorted one with flags that match.

### Items and their mappings

Decays may be expressed in terms of "generic" particles: pions, leptons, etc. More generally, every "thing" in a decay is a *PDG item*. Some items correspond to specific particle states (MC IDs), while others correspond to other (specific) items, either mapping to one particular item (a *shortcut* or *alias*) or mapping to multiple (a *generic* item). The database defines this map between items. 

## Exercises

### Getting a particle

Particles are represented by the `PdgParticle` class. There are multiple ways to get a particle from the API. Depending on the method, the result can be a `PdgParticleList`, a generic `PdgParticle`, or a specific `PdgParticle`.

#### By name

A particle name can refer to a single particle or to a group of them. The function `get_particle_by_name` will return a `PdgParticle` if there's a unique match, and will raise an exception if not. The function `get_particles_by_name` always returns a list of `PdgParticles`.

In [14]:
api.get_particle_by_name('pi+')

PdgParticle('S008/2024', name='pi+')

In [5]:
api.get_particles_by_name('pi')

[PdgParticle('S008/2024', name='pi-'),
 PdgParticle('S009/2024', name='pi0'),
 PdgParticle('S008/2024', name='pi+')]

In [6]:
api.get_particles_by_name('N')

[PdgParticle('S017/2024', name='n'),
 PdgParticle('S016/2024', name='p'),
 PdgParticle('S017/2024', name='nbar'),
 PdgParticle('S016/2024', name='pbar')]

In [13]:
api.get_particles_by_name('Xi_c(2980)')

[PdgParticle('B130/2024', name='Xi_c(2970)0'),
 PdgParticle('B130/2024', name='Xibar_c(2970)0')]

In [11]:
api.get_particles_by_name('Xi_c(2970)')

[PdgParticle('B130/2024', name='Xi_c(2970)0'),
 PdgParticle('B130/2024', name='Xibar_c(2970)0')]

#### By PDG ID

The `get` function, given a particle's PDG ID, returns a list of all associated particles. Since `get` must return an instance of a `PdgData` subclass, the return type is a `PdgParticleList`, rather than a simple list of `PdgParticle`s (as returned by `get_particles_by_name`). 

In [16]:
api.get('S008/2024')

PdgParticleList('S008/2024')

However, a simple list is simple to get:

In [17]:
list(_)

[PdgParticle('S008/2024', name='pi+'), PdgParticle('S008/2024', name='pi-')]

#### By MC ID

In [ ]:
p = api.get_particle_by_mcid(2212)
p

In [ ]:
p.name

### Getting basic properties

### Getting masses, widths, lifetimes

### Getting limits on things

### Getting a particle's decays

### Iterating over decay products

### Getting arbitrary properties

#### Particle-associated properties

#### Particle-independent properties

##### Neutrino properties

##### CKM elements

### Getting all of the things

#### Particles

#### Decays

## Fancier exercises

### Getting all particles with nonzero strangeness

### Printing all decays that produce a $J/\psi$

### Plotting masses of all decay products of $\Upsilon(4S)$

### Plotting precision of \<some_measurement\> over time

### Getting all isospin partners of \<some_particle\>

## Direct access using SQLAlchemy

## Metadata and documentation tables